In [1]:
set -o xtrace
set -e

cd /homes/sb2743/2024-borrowing/lean4/1-runs/run-2023-03-29---10-27---tcg40
echo "pwd: $(pwd)"
DATE=$(date)
echo "date: $DATE"
MACHINE=$(uname -a)
echo "machine: $MACHINE"
echo "git status: $(git status --short)"
echo "git commit: $(git rev-parse HEAD)"
ROOT=$(git rev-parse --show-toplevel)
echo "root folder: $ROOT"
OUTFOLDER=$(pwd)
echo "out folder: $OUTFOLDER"

if [ "$(uname -s)" = "Darwin" ]; then
    TIME="gtime"
else
    TIME="command time"
fi
echo "time: $TIME"
$TIME -v echo "time"

+ set -e
+ cd /homes/sb2743/2024-borrowing/lean4/1-runs/run-2023-03-29---10-27---tcg40
++ pwd
+ echo 'pwd: /homes/sb2743/2024-borrowing/lean4/1-runs/run-2023-03-29---10-27---tcg40'
pwd: /homes/sb2743/2024-borrowing/lean4/1-runs/run-2023-03-29---10-27---tcg40
++ date
+ DATE='Fri 29 Mar 11:28:20 GMT 2024'
+ echo 'date: Fri 29 Mar 11:28:20 GMT 2024'
date: Fri 29 Mar 11:28:20 GMT 2024
++ uname -a
+ MACHINE='Linux dev-cpu-tcg40 6.5.0-26-generic #26~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Mar 12 10:22:43 UTC 2 x86_64 x86_64 x86_64 GNU/Linux'
+ echo 'machine: Linux dev-cpu-tcg40 6.5.0-26-generic #26~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Mar 12 10:22:43 UTC 2 x86_64 x86_64 x86_64 GNU/Linux'
machine: Linux dev-cpu-tcg40 6.5.0-26-generic #26~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Mar 12 10:22:43 UTC 2 x86_64 x86_64 x86_64 GNU/Linux
++ git status --short
+ echo 'git status: ?? ./
?? ../../nohup.out
?? ../../src/Lean/Compiler/IR/ResetReuse.lean'
git status: ?? ./
?? ../../nohup.out
?? ../../src/Le

## Benchmarking Strategy
- Bench on Linux.
- Bench stage2, stage3.
- Copy different variants of reset/reuse file that sets up.
- Disable fine-grained runtime stats to avoid atomics overhead.

In [4]:
rm -rf $ROOT/build || true
rm -rf $ROOT/build/incorrect || true
mkdir -p $ROOT/build/incorrect
cd $ROOT/build/incorrect

# == NO REUSE ==

echo "@@@ NOREUSE BUILD @@@"

CSVNAME=$(date +'%s---%d-%m-%Y---%H:%M:%S')
echo "CSV name is: $CSVNAME"
echo "output file is: OUTFOLDER/$CSVNAME"

cp ../../src/Lean/Compiler/IR/ResetReuse.baseline.lean.in ../../src/Lean/Compiler/IR/ResetReuse.lean
cmake ../../ \
  -DCCACHE=OFF \
  -DRUNTIME_STATS=OFF \
  -DLEAN_RESEARCH_IS_REUSE_ACROSS_CONSTRUCTORS_ENABLED=0 \
  -DLEAN_RESEARCH_COMPILER_PROFILE_CSV_PATH=$OUTFOLDER/$CSVNAME

make -j stage1 2>1 >> log.txt
make -j stage2 2>1 >> log.txt
touch ../../src/Init/Prelude.lean
touch $OUTFOLDER/$CSVNAME && echo "" > $OUTFOLDER/$CSVNAME
$TIME -v make -j stage2 &> "time-noreuse-stage2.txt"
mv $OUTFOLDER/$CSVNAME $OUTFOLDER/$CSVNAME.noreuse.stage2.csv
# $TIME -v make -j stage3 &> "time-noreuse-stage3.txt"
# mv $OUTFOLDER/$CSVNAME $OUTFOLDER/$CSVNAME.noreuse.stage3.csv

curl -d "Done[noreuse]\n*machine:$(uname -a)\n*folder:$OUTFOLDER\n*machine:$MACHINE" ntfy.sh/xISSztEV8EoOchM2
# == REUSE ==

echo "@@@ REUSE BUILD @@@"

# incorrect: don't bother changing directories.
# This maybe can reuse stale data across runs.
# rm -rf $ROOT/build/reuse || true
# mkdir -p $ROOT/build/reuse
# cd $ROOT/build/reuse

CSVNAME="$(date +'%s---%d-%m-%Y---%H:%M:%S')"
echo "output file is: $OUTFOLDER/$CSVNAME"

cp ../../src/Lean/Compiler/IR/ResetReuse.research.lean.in ../../src/Lean/Compiler/IR/ResetReuse.lean

cmake ../../ \
  -DCCACHE=OFF \
  -DRUNTIME_STATS=OFF \
  -DLEAN_RESEARCH_IS_REUSE_ACROSS_CONSTRUCTORS_ENABLED=1 \
  -DLEAN_RESEARCH_COMPILER_PROFILE_CSV_PATH=$OUTFOLDER/$CSVNAME
make -j stage1  2>1 >> log.txt
make -j stage2  2>1 >> log.txt
touch ../../src/Init/Prelude.lean
touch $OUTFOLDER/$CSVNAME && echo "" > $OUTFOLDER/$CSVNAME
$TIME -v make -j stage2 &> "time-reuse-stage2.txt"
mv $OUTFOLDER/$CSVNAME $OUTFOLDER/$CSVNAME.reuse.stage2.csv

# $TIME -v make -j stage3 &> "time-reuse-stage3.txt"
# mv $OUTFOLDER/$CSVNAME $OUTFOLDER/$CSVNAME.reuse.stage3.csv

echo "DONE!"
curl -d "Done fully\n*machine:$(uname -a)\n*folder:$OUTFOLDER\n*machine:$MACHINE" ntfy.sh/xISSztEV8EoOchM2

+ rm -rf /auto/homes/sb2743/2024-borrowing/lean4/build
+ rm -rf /auto/homes/sb2743/2024-borrowing/lean4/build/noreuse
+ mkdir -p /auto/homes/sb2743/2024-borrowing/lean4/build/noreuse
+ cd /auto/homes/sb2743/2024-borrowing/lean4/build/noreuse
+ echo '@@@ NOREUSE BUILD @@@'
@@@ NOREUSE BUILD @@@
++ date +%s---%d-%m-%Y---%H:%M:%S
+ CSVNAME=1711710923---29-03-2024---11:15:23
+ echo 'CSV name is: 1711710923---29-03-2024---11:15:23'
CSV name is: 1711710923---29-03-2024---11:15:23
+ echo 'output file is: OUTFOLDER/1711710923---29-03-2024---11:15:23'
output file is: OUTFOLDER/1711710923---29-03-2024---11:15:23
+ cp ../../src/Lean/Compiler/IR/ResetReuse.baseline.lean.in ../../src/Lean/Compiler/IR/ResetReuse.lean
+ cmake ../../ -DCCACHE=OFF -DRUNTIME_STATS=OFF -DLEAN_RESEARCH_IS_REUSE_ACROSS_CONSTRUCTORS_ENABLED=0 -DLEAN_RESEARCH_COMPILER_PROFILE_CSV_PATH=/homes/sb2743/2024-borrowing/lean4/1-runs/run-2023-03-29---10-27---tcg40/1711710923---29-03-2024---11:15:23
-- The CXX compiler identification

In [7]:
# tail -f $ROOT/build/reuse/time-noreuse-stage2.txt -n 35
# tail -f $ROOT/build/noreuse/time-noreuse-stage2.txt -n 35
# tail -f $ROOT/build/reuse/time-noreuse-stage2.txt -n 35
# tail -f $ROOT/build/noreuse/time-noreuse-stage2.txt -n 35
cp $ROOT/build/noreuse/time-noreuse-stage2.txt  $OUTFOLDER/time-noreuse-stage2.txt
cp $ROOT/build/noreuse/time-noreuse-stage3.txt  $OUTFOLDER/time-noreuse-stage3.txt

cp $ROOT/build/reuse/time-reuse-stage2.txt  $OUTFOLDER/time-reuse-stage2.txt  # TODO: fix this typo!
cp $ROOT/build/reuse/time-reuse-stage3.txt  $OUTFOLDER/time-reuse-stage3.txt